In [ ]:
%pip install bitshuffle

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import blimpy as bl
#from ultralytics import YOLO
import time
import os
import random
import psutil
import scipy.ndimage
from PIL import Image
from PIL import ImageDraw
%matplotlib inline

In [ ]:
# import cv2

# # Load the image
# image = cv2.imread('/home/jliang/gbt-rfi/example.png', cv2.IMREAD_UNCHANGED)

# # Print shape and number of channels
# print(f"Shape: {image.shape}")

# if len(image.shape) == 2:
#     print("Image is grayscale (1 channel).")
# elif len(image.shape) == 3:
#     print(f"Image has {image.shape[2]} channels.")
# else:
#     print("Unexpected image format.")


In [ ]:
df = pd.read_csv('/home/cgchoza/galaxies/complete_cadences_catalog.csv')
df

In [ ]:
high_path = df['.h5 path'].iloc[-1]

In [ ]:
df['.h5 path'] = df['.h5 path'].str.replace('0000.h5', '0002.h5', regex=False)
df = df.drop_duplicates(subset='.h5 path', keep='first').reset_index(drop=True)
df

In [ ]:
df = df[~df['.h5 path'].str.contains('spliced')].reset_index(drop=True)
df

In [ ]:
#path = df['.h5 path'].iloc[-1]

In [ ]:
# fb_high = bl.Waterfall(high_path)
# high_data = fb_high.data # shape: (279; 1; 65,536)
# plt.imshow(10*np.log10(high_data).squeeze(), aspect='auto')
# #plt.xlim(1250, 1251)
# plt.show()

In [ ]:
# fb_mid = bl.Waterfall(path)
# mid_data = fb_mid.data # shape: (279; 1; 65,536)
# plt.imshow(10*np.log10(mid_data).squeeze(), aspect='auto')
# plt.xlim(10000, 10003)
# plt.show()

In [ ]:
# fb_mid = bl.Waterfall(path)
# mid_data = fb_mid.data # shape: (279; 1; 65,536)
# plt.imshow(10*np.log10(mid_data).squeeze(), aspect='auto')
# plt.xlim(1250, 1251)
# plt.show()

In [ ]:
df = df.drop(index = 17546)
small_df = df.sample(n=100, random_state=42).reset_index(drop=True)

In [ ]:
# import blimpy as bl
# import numpy as np
# import scipy.ndimage
# from PIL import Image
# import matplotlib.pyplot as plt
# from PIL import Image, ImageDraw

# factor = 7

# def get_mask(path, factor=7):

#     # Load data
#     fb = bl.Waterfall(path, load_data=True)
#     data = fb.data.squeeze()  # shape: (time, freq)
#     data = 10*np.log10(data)

#     # Mask = keep everything
#     mask = np.ones_like(data, dtype=bool)

#     # Stats from masked data
#     # Define coarse channel width (depends on your setup; usually 1024 or 128)
#     coarse_channel_width = 128  # change if needed
#     num_coarse = data.shape[1] // coarse_channel_width

#     # Mask out edge channels from each coarse block
#     bad_channels = []
#     for i in range(num_coarse):
#         edge_bins = list(range(i * coarse_channel_width, i * coarse_channel_width + 5))  # 5-bin edge mask
#         bad_channels.extend(edge_bins)

#     # Apply masking
#     data[:, bad_channels] = np.median(data)

#     masked_values = data[mask]
#     median = np.median(masked_values)
#     std = np.std(masked_values)

#     # Threshold to identify hits
#     threshold = median + factor * std
#     binary = data > threshold  # not mask
#     # Merge nearby frequency hits (widen horizontally)
#     # Fast dilation using a max filter (approximates dilation)
#     binary = scipy.ndimage.maximum_filter(binary, size=(10, 300))


#     # Label connected regions
#     labeled, num_features = scipy.ndimage.label(binary)
#     slices = scipy.ndimage.find_objects(labeled)

#     # Set YOLO output image size
#     target_w, target_h = 640, 640

#     # Original image dimensions
#     orig_h, orig_w = data.shape

#     # Bounding boxes
#     labels = []

#     for sl in slices:
#         t0, t1 = sl[0].start, sl[0].stop  # time axis
#         f0, f1 = sl[1].start, sl[1].stop  # frequency axis

#         # Rescale to resized image dimensions
#         x_center = ((f0 + f1) / 2) * target_w / orig_w
#         y_center = ((t0 + t1) / 2) * target_h / orig_h
#         #h = 1.0         # full height
#         w = (f1 - f0) * target_w / orig_w
#         h = (t1 - t0) * target_h / orig_h

#         # Normalize for YOLO format (0–1 range)
#         x_center /= target_w
#         y_center /= target_h
#         w /= target_w
#         h /= target_h

#         labels.append(f"0 {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}")


#     # Save labels
#     with open("example.txt", "w") as f:
#         f.write("\n".join(labels))

#     # Save image from data
#     img = (255 * (data - data.min()) / data.ptp()).astype(np.uint8)
#     img = Image.fromarray(img)

#     # Resize to YOLO-compatible shape (e.g., 640x640)
#     img = img.resize((640, 640), Image.BICUBIC)
#     img.save(f"example_threshold{factor}.png")

# def draw_boxes(image_path, label_path):
#     ## ----------- Visualization ------------##

#     # Load image
#     img = Image.open(image_path).convert("RGB")
#     draw = ImageDraw.Draw(img)
#     w, h = img.size

#     # Load and draw bounding boxes
#     with open(label_path, "r") as f:
#         for line in f:
#             parts = line.strip().split()
#             if len(parts) != 5:
#                 continue  # skip malformed lines
#             cls, x_center, y_center, box_w, box_h = map(float, parts)

#             # Convert from normalized to pixel coordinates
#             x_center *= w
#             y_center *= h
#             box_w *= w
#             box_h *= h

#             x0 = int(x_center - box_w / 2)
#             y0 = int(y_center - box_h / 2)
#             x1 = int(x_center + box_w / 2)
#             y1 = int(y_center + box_h / 2)

#             # Draw rectangle (red box)
#             draw.rectangle([x0, y0, x1, y1], outline="red", width=2)

#     # Save or display the result
#     img.save(f"{image_path}_with_boxes.png")
#     img.show()

# Optional visualization
# plt.imshow(data.T, origin='lower', aspect='auto')
# for sl in slices:
#     t0, t1 = sl[0].start, sl[0].stop
#     f0, f1 = sl[1].start, sl[1].stop
#     plt.gca().add_patch(plt.Rectangle((t0, f0), t1 - t0, f1 - f0,
#                                       edgecolor='red', facecolor='none', lw=1))
# plt.title("Detected Hits")
# plt.show()



In [ ]:
# problem_path = small_df['.h5 path'].iloc[692]
# print(df[df['.h5 path'] == problem_path].index)

In [ ]:
# fb = bl.Waterfall(problem_path, load_data=True)
# fb.plot_waterfall(cmap='viridis', figsize=(10, 5), title='Waterfall Plot')

In [ ]:
def yolo_to_corners(xc, yc, w, h):
    x0 = xc - w / 2
    x1 = xc + w / 2
    y0 = yc - h / 2
    y1 = yc + h / 2
    return x0, y0, x1, y1

def is_inside(inner, outer, eps=1e-6):
    ix0, iy0, ix1, iy1 = inner
    ox0, oy0, ox1, oy1 = outer
    return (ix0 >= ox0 - eps and iy0 >= oy0 - eps and
            ix1 <= ox1 + eps and iy1 <= oy1 + eps)

def filter_nested_boxes(yolo_labels):
    parsed = []
    for s in yolo_labels:
        cls, xc, yc, w, h = map(float, s.strip().split())
        corners = yolo_to_corners(xc, yc, w, h)
        parsed.append((cls, xc, yc, w, h, corners))
    keep = []
    n = len(parsed)
    for i, box in enumerate(parsed):
        inner = box[5]
        is_nested = False
        for j, other in enumerate(parsed):
            if i == j:
                continue
            outer = other[5]
            if is_inside(inner, outer):
                is_nested = True
                break
        if not is_nested:
            keep.append(f"{int(box[0])} {box[1]:.6f} {box[2]:.6f} {box[3]:.6f} {box[4]:.6f}")
    return keep


In [ ]:
# Shuffle and split
total_indices = list(range(len(small_df)))
random.seed(42)
random.shuffle(total_indices)

split_idx = int(0.8 * len(total_indices))
train_indices = set(total_indices[:split_idx])
val_indices = set(total_indices[split_idx:])

# Create all needed dirs
base_dir = '/datax/scratch/jliang/masking_dataset_noresize'
image_train_dir = os.path.join(base_dir, 'train/images')
label_train_dir = os.path.join(base_dir, 'train/labels')
image_val_dir = os.path.join(base_dir, 'val/images')
label_val_dir = os.path.join(base_dir, 'val/labels')
visualization_train_dir = os.path.join(base_dir, 'visualization/train')
visualization_val_dir = os.path.join(base_dir, 'visualization/val')
waterfall_train_dir = os.path.join(base_dir, 'waterfall/train')
waterfall_val_dir = os.path.join(base_dir, 'waterfall/val')

for d in [image_train_dir, label_train_dir, image_val_dir, label_val_dir]:
    os.makedirs(d, exist_ok=True)

# Factor for thresholding
factor = 7

In [ ]:
import multiprocessing as mp

def process_file(index, row, train_indices, base_dirs):
    # Unpack base directories
    image_train_dir, label_train_dir, image_val_dir, label_val_dir, visualization_train_dir, visualization_val_dir, waterfall_train_dir, waterfall_val_dir = base_dirs

    # Choose correct folder based on split
    if index in train_indices:
        image_dir = image_train_dir
        label_dir = label_train_dir
        vis_dir = visualization_train_dir
        w_dir = waterfall_train_dir
    else:
        image_dir = image_val_dir
        label_dir = label_val_dir
        vis_dir = visualization_val_dir
        w_dir = waterfall_val_dir

    h5_path = row['.h5 path']

    try:
        # Load data
        fb = bl.Waterfall(h5_path, load_data=True)
        data = fb.data.squeeze()  # shape: (time, freq)
        data = 10 * np.log10(data)

        # Mask = keep everything
        mask = np.ones_like(data, dtype=bool)

        # Stats from masked data
        coarse_channel_width = 1024  # change if needed
        num_coarse = data.shape[1] // coarse_channel_width

        # Mask out edge channels from each coarse block
        bad_channels = []
        for i in range(num_coarse):
            edge_bins = list(range(i * coarse_channel_width, i * coarse_channel_width + 5))  # 5-bin edge mask
            bad_channels.extend(edge_bins)

        # Apply masking
        data[:, bad_channels] = np.median(data)

        masked_values = data[mask]
        median = np.median(masked_values)
        std = np.std(masked_values)

        # Threshold to identify hits
        threshold = median + factor * std
        binary = data > threshold  # not mask
        binary = scipy.ndimage.maximum_filter(binary, size=(10, 300))

        # Label connected regions
        labeled, num_features = scipy.ndimage.label(binary)
        slices = scipy.ndimage.find_objects(labeled)

        orig_h, orig_w = data.shape

        labels = []
        for sl in slices:
            t0, t1 = sl[0].start, sl[0].stop
            f0, f1 = sl[1].start, sl[1].stop

            # Use original coordinates (no scaling needed)
            x_center = (f0 + f1) / 2 / orig_w
            y_center = (t0 + t1) / 2 / orig_h
            w = (f1 - f0) / orig_w
            h = (t1 - t0) / orig_h

            labels.append(f"0 {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}")

        labels = filter_nested_boxes(labels)

        # Save labels
        label_name = f'img_{index:05d}.txt'
        label_path = os.path.join(label_dir, label_name)
        with open(label_path, "w") as f:
            f.write("\n".join(labels))

        # Normalize to 8-bit grayscale
        img = (255 * (data - data.min()) / data.ptp()).astype(np.uint8)
        img = np.stack([img] * 3, axis=-1)  # shape: (H, W, 3)
        img = Image.fromarray(img)

        # Save as 3-channel RGB PNG
        img_name = f'img_{index:05d}.png'
        out_path = os.path.join(image_dir, img_name)
        img.save(out_path)

        # Visualization
        img = Image.open(out_path).convert("RGB")
        draw = ImageDraw.Draw(img)
        w, h = img.size

        with open(label_path, "r") as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) != 5:
                    continue
                cls, x_center, y_center, box_w, box_h = map(float, parts)
                x_center *= w
                y_center *= h
                box_w *= w
                box_h *= h
                x0 = int(x_center - box_w / 2)
                y0 = int(y_center - box_h / 2)
                x1 = int(x_center + box_w / 2)
                y1 = int(y_center + box_h / 2)
                draw.rectangle([x0, y0, x1, y1], outline="red", width=2)

        vis_path = os.path.join(vis_dir, f"img_{index:05d}.png")
        img.save(vis_path)

        # Save waterfall plot
        w_data = fb.data.squeeze()
        w_data = 10 * np.log10(w_data)
        w_name = f'img_{index:05d}.jpg'
        w_path = os.path.join(w_dir, w_name)
        plt.figure(figsize=(6.4, 6.4), dpi=100)
        plt.imshow(w_data, aspect='auto', cmap='viridis')
        plt.axis('off')
        plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
        plt.savefig(w_path, bbox_inches='tight', pad_inches=0)
        plt.close()

        return index, None  # No error

    except Exception as e:
        return index, str(e)  # Return error message


# Parallel processing
if __name__ == "__main__":
    num_workers = mp.cpu_count()  # Use all available CPUs
    pool = mp.Pool(num_workers)

    base_dirs = (
        image_train_dir, label_train_dir, image_val_dir, label_val_dir,
        visualization_train_dir, visualization_val_dir, waterfall_train_dir, waterfall_val_dir
    )

    results = pool.starmap(
        process_file,
        [(index, row, train_indices, base_dirs) for index, row in small_df.iterrows()]
    )

    pool.close()
    pool.join()

    # Handle results
    for index, error in results:
        if error:
            print(f"Error processing index {index}: {error}")

In [ ]:
# # Shuffle and split
# total_indices = list(range(len(small_df)))
# random.seed(42)
# random.shuffle(total_indices)

# split_idx = int(0.8 * len(total_indices))
# train_indices = set(total_indices[:split_idx])
# val_indices = set(total_indices[split_idx:])

# # Create all needed dirs
# base_dir = '/datax/scratch/jliang/masking_dataset_noresize'
# image_train_dir = os.path.join(base_dir, 'train/images')
# label_train_dir = os.path.join(base_dir, 'train/labels')
# image_val_dir = os.path.join(base_dir, 'val/images')
# label_val_dir = os.path.join(base_dir, 'val/labels')
# visualization_train_dir = os.path.join(base_dir, 'visualization/train')
# visualization_val_dir = os.path.join(base_dir, 'visualization/val')
# waterfall_train_dir = os.path.join(base_dir, 'waterfall/train')
# waterfall_val_dir = os.path.join(base_dir, 'waterfall/val')

# for d in [image_train_dir, label_train_dir, image_val_dir, label_val_dir]:
#     os.makedirs(d, exist_ok=True)

# # Factor for thresholding
# factor = 7

# empty_label_count = 0
# empty_label_names = []

# for index in range(0, len(small_df)):
#     # Wait for memory if needed
#     mem_threshold = 20 * (1024 ** 3)  # 20 GB
#     while psutil.virtual_memory().available < mem_threshold:
#         print('Waiting for memory to free up...')
#         time.sleep(180)

#     # Choose correct folder based on split
#     if index in train_indices:
#         image_dir = image_train_dir
#         label_dir = label_train_dir
#     else:
#         image_dir = image_val_dir
#         label_dir = label_val_dir
#     h5_path = small_df['.h5 path'].iloc[index]
    
#     # Load data
#     fb = bl.Waterfall(h5_path, load_data=True)
#     data = fb.data.squeeze()  # shape: (time, freq)
#     data = 10*np.log10(data)

#     # Mask = keep everything
#     mask = np.ones_like(data, dtype=bool)

#     # Stats from masked data
#     # Define coarse channel width (depends on your setup; usually 1024 or 128)
#     coarse_channel_width = 1024  # change if needed
#     num_coarse = data.shape[1] // coarse_channel_width

#     # Mask out edge channels from each coarse block
#     bad_channels = []
#     for i in range(num_coarse):
#         edge_bins = list(range(i * coarse_channel_width, i * coarse_channel_width + 5))  # 5-bin edge mask
#         bad_channels.extend(edge_bins)

#     # Apply masking
#     data[:, bad_channels] = np.median(data)

#     masked_values = data[mask]
#     median = np.median(masked_values)
#     std = np.std(masked_values)

#     # Threshold to identify hits
#     threshold = median + factor * std
#     binary = data > threshold  # not mask
#     # Merge nearby frequency hits (widen horizontally)
#     # Fast dilation using a max filter (approximates dilation)
#     binary = scipy.ndimage.maximum_filter(binary, size=(10, 300))


#     # Label connected regions
#     labeled, num_features = scipy.ndimage.label(binary)
#     slices = scipy.ndimage.find_objects(labeled)

#     orig_h, orig_w = data.shape

#     labels = []
#     for sl in slices:
#         t0, t1 = sl[0].start, sl[0].stop
#         f0, f1 = sl[1].start, sl[1].stop

#         # Use original coordinates (no scaling needed)
#         x_center = (f0 + f1) / 2 / orig_w
#         y_center = (t0 + t1) / 2 / orig_h
#         w = (f1 - f0) / orig_w
#         h = (t1 - t0) / orig_h

#         labels.append(f"0 {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}")

#     labels = filter_nested_boxes(labels)


#     # Save labels
#     label_name = f'img_{index:05d}.txt'
#     label_path = os.path.join(label_dir, label_name)
#     with open(label_path, "w") as f:
#         f.write("\n".join(labels))
#     if len(labels) == 0:
#         empty_label_count += 1
#         empty_label_names.append(label_name)

#      # Normalize to 8-bit grayscale
#     img = (255 * (data - data.min()) / data.ptp()).astype(np.uint8)

#     # Stack into 3 channels to simulate RGB
#     img = np.stack([img] * 3, axis=-1)  # shape: (H, W, 3)

#     # Convert to PIL image and resize
#     img = Image.fromarray(img)

#     # Save as 3-channel RGB PNG
#     img_name = f'img_{index:05d}.png'
#     out_path = os.path.join(image_dir, img_name)
#     img.save(out_path)


#     ## ----------- Visualization ------------##

#     # Load image
#     img = Image.open(out_path).convert("RGB")
#     draw = ImageDraw.Draw(img)
#     w, h = img.size

#     # Load and draw bounding boxes
#     with open(label_path, "r") as f:
#         for line in f:
#             parts = line.strip().split()
#             if len(parts) != 5:
#                 continue  # skip malformed lines
#             cls, x_center, y_center, box_w, box_h = map(float, parts)

#             # Convert from normalized to pixel coordinates
#             x_center *= w
#             y_center *= h
#             box_w *= w
#             box_h *= h

#             x0 = int(x_center - box_w / 2)
#             y0 = int(y_center - box_h / 2)
#             x1 = int(x_center + box_w / 2)
#             y1 = int(y_center + box_h / 2)

#             # Draw rectangle (red box)
#             draw.rectangle([x0, y0, x1, y1], outline="red", width=2)

#     # Save or display the result
#     vis_dir = visualization_train_dir if index in train_indices else visualization_val_dir
#     os.makedirs(vis_dir, exist_ok=True)  # just in case
#     vis_path = os.path.join(vis_dir, f"img_{index:05d}.png")
#     img.save(vis_path)

#     w_data = fb.data  # shape: (time, 1, freq)
#     w_data = 10 * np.log10(w_data).squeeze()  # shape: (time, freq)

#     # Set up save path
#     w_name = f'img_{index:05d}.jpg'
#     w_dir = waterfall_train_dir if index in train_indices else waterfall_val_dir
#     os.makedirs(w_dir, exist_ok=True)
#     w_path = os.path.join(w_dir, w_name)

#     # Plot and save
#     plt.figure(figsize=(6.4, 6.4), dpi=100)
#     plt.imshow(w_data, aspect='auto', cmap='viridis')  # origin='lower' makes freq go up
#     plt.axis('off')
#     plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
#     plt.savefig(w_path, bbox_inches='tight', pad_inches=0)
#     plt.close()

#     print("Processed index:", index)
    
# # Save summary of empty labels
# txt_output_path = os.path.join(base_dir, "empty_labels_summary.txt")
# with open(txt_output_path, "w") as txt_file:
#     txt_file.write(f"Total files with empty labels: {empty_label_count} out of {len(small_df)}\n\n")
#     txt_file.write("Empty label files:\n")
#     for name in empty_label_names:
#         txt_file.write(f"- {name}\n")
